In [12]:
import random
import pandas as pd

from ME_BPER import ME_BPER
#from ME_IPER import ME_IPER
from ME_LOC import add_location
from ME_ORG import add_organisation

ImportError: cannot import name 'add_organisation' from 'ME_ORG' (/Users/norazouhtane/Downloads/nlp25_project_gr2/data_aug_code/ME_ORG.py)

In [6]:
df = pd.read_csv("../data_aug_sources/the-middle-east-cities.csv", sep = ";", skiprows = 1)
unique_city_da = df["city_da"].drop_duplicates()
ME_LOC = [add_location(loc) for loc in unique_city_da]

In [13]:
df = pd.read_csv("../data_aug_sources/middle_eastern_organisations.csv", sep = ";", skiprows = 1)
unique_orgs = df["org"].drop_duplicates()
ME_ORG = [add_organisation(org) for org in unique_orgs]

NameError: name 'add_organisation' is not defined

In [11]:
ME_BPER

['Bilal',
 'R',
 'Ikhlas*',
 'Shafiq*',
 'Afifa*',
 'Faysal',
 'Balsam*',
 'Dalia',
 'Shatha*',
 'Bashir',
 'Mushtaq',
 'Hadil',
 'Saja',
 'Kamila',
 'Süleyman',
 'Shirin',
 'Nail',
 'Fadi',
 'Faiz*',
 'Khadim*',
 'Navid',
 'Samad',
 'Sheikh',
 'Umar',
 'Nusrat',
 'Ejaz*',
 'Qasim',
 'Younas*',
 'Asiya',
 'Nidal*',
 'Humaira*',
 'Baker*',
 'Abdo*',
 'Arshad',
 'Aziz',
 'Asmaa',
 'Farouk*',
 'Manna',
 'Nura',
 'Ayah',
 'Salik*',
 'Tahsin',
 'Sajeda*',
 'Hanna',
 'Samiha*',
 'Hazem',
 'Idriz',
 'Suha*',
 'Abdou*',
 'Karam',
 'Ghadir*',
 'Fayza*',
 'Abdallah',
 'Alim*',
 'Nazmi',
 'Talha',
 'Amina',
 'Mir',
 'Sinan',
 'Hajar',
 'Mirah*',
 'Jabbar*',
 'Safia',
 'Surayya*',
 'Nayef’',
 'Sulaiman',
 'Ayub',
 'Dana',
 'Nazim',
 'Fadia',
 'Abdulahi',
 'Hadia*',
 'Hafsa',
 'Zahida',
 'Najiba',
 'Sahar',
 'Ashraf',
 'Fadl*',
 'Saman',
 'Salman',
 'Numan*',
 'Rizwan',
 'Mounir',
 'Ansar*',
 'Ghaleb*',
 'W',
 'Khadijah*',
 'Hafiza*',
 'Afif*',
 'Hamad',
 'Batul*',
 'Rasul*',
 'Fida*',
 'Susan',
 '

In [ ]:
def data_aug_replace(dataset):
    for sent in dataset:
        i = 0

        while i<len(sent["tokens"]):
            tag = sent["ner_tags"][i]

            if tag == 'B-PER':
                replace = random.choice(ME_BPER)
                sent["ner_tags"][i] = replace
                ME_BPER.remove(replace)
                i+=1

            elif tag == 'I-PER':
                replace = random.choice(ME_IPER)
                sent["ner_tags"][i] = replace
                ME_IPER.remove(replace)
                i+=1

            elif tag == 'B-LOC':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-LOC":
                    span_len += 1
                    i += 1

                same_length_LOC = []
                for loc in ME_LOC:
                    if len(loc["tokens"]) == span_len:
                        same_length_LOC.append(loc)
                if same_length_LOC:
                    replace = random.choice(same_length_LOC)
                    sent["tokens"][span_start:span_start+span_len] = replace
                    ME_LOC.remove(replace)
            
            elif tag == 'B-ORG':
                span_start = i
                span_len = 1
                i += 1
                while i < len(sent["ner_tags"]) and sent["ner_tags"][i] == "I-ORG":
                    span_len += 1
                    i += 1

                same_length_ORG = []
                for org in ME_ORG:
                    if len(org["tokens"]) == span_len:
                        same_length_ORG.append(org)
                if same_length_ORG:
                    replace = random.choice(same_length_ORG)
                    sent["tokens"][span_start:span_start+span_len] = replace
                    ME_ORG.remove(replace)

            else:
                i+=1
